# 영화 추천 시스템 프로젝트

## - 목차

1. 데이터 준비 및 전처리
2. 내가 선호하는 영화 추가
3. 모델에 활용하기 위한 전처리
4. CSR Matrix 생성
5. 모델 훈련
6. 모델 예측
7. 영화 추천
- 프로젝트 정리

## 1. 데이터 준비 및 전처리

In [1]:
import pandas as pd
import os

#영화 평점 파일 로드
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
#영화 제목 파일 로드
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
#평점 3점 이상만 남김
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
#rating 컬럼의 이름을 count로 변경
#ratings.rename(columns={'rating':'count'}, inplace=True)
#ratings['count']

- 노드에서는 위의 코드를 실행했는데, 평점을 왜 count로 바꾸는지 이해가 되지 않았다.
- count라는 변수명은 영화를 시청한 횟수를 의미하는 것 같은데 해당 칼럼의 값의 최대값이 5이고, 시청한 횟수를 5번으로 제한하는 것 보다 평점을 최대 5점으로 하는 것이 더 자연스럽다고 판단했다.
- 때문에 노드에서는 rating을 count로 바꾸었지만 나는 그대로 rating으로 두고 해당 칼럼의 의미를 평점으로 두고 진행을 했다.

In [5]:
#ratings에서 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [6]:
#ratings에서 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [7]:
#ratings에서 유니크한 영화 제목 수 
movies['title'].nunique()

3883

In [8]:
#평점과 제목 데이터프레임 합치기
movie_data = pd.merge(ratings, movies, how = 'left', on = 'movie_id')
movie_data.head()

,user_id,movie_id,rating,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [9]:
#사용하지 않는 컬럼 제거
movie_data = movie_data.drop(columns=['timestamp', 'genre','movie_id'])
movie_data.head()

,user_id,rating,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,1,3,James and the Giant Peach (1996)
2,1,3,My Fair Lady (1964)
3,1,4,Erin Brockovich (2000)
4,1,5,"Bug's Life, A (1998)"


In [10]:
#가장 인기있는 영화 30개(인기순)
movie_count = movie_data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
#유저 별 본 영화 편 수 통계
user_count = movie_data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 2. 내가 선호하는 영화 추가

In [16]:
#내가 좋아하는 영화 5편 리스트 만들기
my_favorite = ['Men in Black (1997)' , 'Contact(1997)' ,'Terminator, The (1984)' ,
               'Toy Story (1995)' ,'Forrest Gump (1994)']

#user_id가 'hyojeong'인 나는 위의 5편의 영화의 평점을 모두 5점 씩 줌.
my_rating = pd.DataFrame({'user_id' : ['hyojeong']*5, 'title' : my_favorite, 'rating' : [5]*5})

#user_id에 'hyojeong'가 없으면 my_favorite 데이터를 기존 데이터에 추가 함.
if not movie_data.isin({'user_id' : ['hyojeong']})['user_id'].any() :
    movie_data = movie_data.append(my_rating, ignore_index = True)
    
movie_data.tail(10)

,user_id,rating,title
836473,6040,3,Platoon (1986)
836474,6040,5,"Crying Game, The (1992)"
836475,6040,5,Welcome to the Dollhouse (1995)
836476,6040,4,Sophie's Choice (1982)
836477,6040,4,E.T. the Extra-Terrestrial (1982)
0,hyojeong,5,Men in Black (1997)
1,hyojeong,5,Contact(1997)
2,hyojeong,5,"Terminator, The (1984)"
3,hyojeong,5,Toy Story (1995)
4,hyojeong,5,Forrest Gump (1994)


## 3. 모델에 활용하기 위한 전처리

In [18]:
#고유한 유저와 영화 리스트 확인
user_unique = movie_data['user_id'].unique()
title_unique = movie_data['title'].unique()

print('user_unique:', user_unique)
print('title_unique:', title_unique)

user_unique: [1 2 3 ... 6039 6040 'hyojeong']
title_unique: ["One Flew Over the Cuckoo's Nest (1975)"
 'James and the Giant Peach (1996)' 'My Fair Lady (1964)' ...
 'Five Wives, Three Secretaries and Me (1998)'
 'Identification of a Woman (Identificazione di una donna) (1982)'
 'Contact(1997)']


In [20]:
#유저, 타이틀 인덱싱
user_idx = {v:k for k,v in enumerate(user_unique)}
title_idx = {v:k for k,v in enumerate(title_unique)}

#인덱싱 확인
print(user_idx['hyojeong']) # 6040명의 유저 중 마지막으로 추가된 유저니까 6039이 나올 것. 
print(title_idx['Forrest Gump (1994)'])

6039
160


In [21]:
#데이터 칼럼 값을 인덱싱 값으로 변경

#user_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구함. 
#혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거. 
temp_user_data = movie_data['user_id'].map(user_idx.get).dropna()
if len(temp_user_data) == len(movie_data):   #모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_data['user_id'] = temp_user_data   #movie_data['user_id']을 인덱싱 된 Series로 교체. 
else:
    print('user_id column indexing Fail!!')

#temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱. 
temp_title_data = movie_data['title'].map(title_idx.get).dropna()
if len(temp_title_data) == len(movie_data):
    print('title column indexing OK!!')
    movie_data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

movie_data

user_id column indexing OK!!
title column indexing OK!!


,user_id,rating,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
...,...,...,...
0,6039,5,175
1,6039,5,3628
2,6039,5,200
3,6039,5,40


## 4. CSR Matrix 생성

In [22]:
movie_data['rating']

0    5
1    3
2    3
3    4
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64

In [24]:
from scipy.sparse import csr_matrix

user_num = movie_data['user_id'].nunique()
movie_num = movie_data['title'].nunique()

csr_data = csr_matrix((movie_data['rating'], (movie_data.user_id, movie_data.title)), shape=(user_num, movie_num))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. 모델 훈련
- AlternatingLeastSquares 모델 생성 후 훈련

In [25]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

#implicit 라이브러리에서 권장하고 있는 부분이며 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [26]:
#Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [27]:
#als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose 해 줌.
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [28]:
#모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 모델 예측

In [31]:
#훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
hyojeong, toystory = user_idx['hyojeong'], title_idx['Toy Story (1995)']
hyojeong_vector, toystory_vector = als_model.user_factors[hyojeong], als_model.item_factors[toystory]

In [32]:
#나의 벡터
hyojeong_vector

array([ 0.28623024, -0.19663216, -0.1953373 ,  0.20308943, -0.40468872,
        0.6876701 , -0.15529266, -0.18045817, -0.86241746, -0.59646356,
       -0.36681944,  0.44173738, -0.22247894, -0.37327778,  0.23792693,
        0.02292245, -0.2349109 ,  0.97089076,  0.0370808 ,  0.4411986 ,
        0.5446322 , -0.00794802, -0.63261646,  0.39124286,  0.6821147 ,
       -0.30501685,  0.1819453 , -0.25392786,  0.7281686 , -0.02808323,
        0.49329555, -0.41108373, -0.10695752, -0.21896851, -0.03782182,
       -0.16528201,  0.3231683 , -0.05231161,  0.18666312, -0.9926605 ,
        0.03322313,  0.2478739 , -0.10359219,  0.5943115 ,  0.29659024,
        0.9788672 , -0.02048217,  0.18725726, -0.21633278, -0.38325202,
       -0.08598775, -0.2097976 , -1.0436068 ,  0.23967297,  0.0794382 ,
        0.58030844, -0.6057632 ,  0.20104472,  0.0685622 , -0.03596087,
       -0.35931072,  0.10814738,  0.2760441 ,  0.4455065 , -0.03836733,
        0.28557572, -0.3109032 , -1.1102569 , -0.38372087, -0.38

In [33]:
#영화 토이 스토리 벡터
toystory_vector

array([ 0.03569599, -0.01427653,  0.00428272, -0.00849583,  0.00337399,
       -0.00088234,  0.00505883,  0.00416068, -0.0270489 , -0.01127137,
       -0.00414098,  0.01842524, -0.00683753, -0.02562911,  0.0156469 ,
       -0.0096097 ,  0.01324109,  0.00375446, -0.02880429,  0.00238672,
        0.00807991, -0.00064144,  0.0084403 ,  0.00389105,  0.03595176,
       -0.00084515,  0.01902527, -0.0159827 ,  0.0518489 , -0.00902912,
        0.02342092,  0.02851536,  0.00399146, -0.00286388,  0.02358123,
        0.03232649,  0.00064334,  0.01766173, -0.00748266, -0.02265515,
        0.00485503,  0.02011907,  0.01924141,  0.0052152 ,  0.02722261,
        0.03621069, -0.00634331, -0.02353211,  0.01781888,  0.01602581,
       -0.01073076,  0.0089189 , -0.03440608, -0.00879237, -0.0122527 ,
        0.02164301, -0.01757217,  0.00246203,  0.01612477,  0.01227866,
       -0.0250059 , -0.00165732,  0.01908525,  0.02456511,  0.01853113,
        0.00647033, -0.0102184 , -0.00238316, -0.02435526,  0.01

### - 나의 선호 리스트에 있는 영화에 대한 훈련된 모델이 예측한 나의 선호도

In [34]:
#나의 벡터와 영화 토이 스토리 벡터 내적
np.dot(hyojeong_vector, toystory_vector)

0.51244164

### - 나의 선호 리스트에 없는 영화에 대한 훈련된 모델이 예측한 나의 선호도

In [37]:
matrix = title_idx['Matrix, The (1999)']
matrix_vector = als_model.item_factors[matrix]
np.dot(hyojeong_vector, matrix_vector)

0.43386862

## 7. 영화 추천

### - 내가 선호하는 영화와 비슷한 영화 추천

In [44]:
#내가 선호하는 영화 5편에 대해 전부 적용해 보고자 영화 추천 함수로 따로 만듦

idx_title = {v:k for k,v in title_idx.items()}

def similar_movie(title_name: str):
    title_id = title_idx[title_name]
    
    #(movie idx, 유사도) 형식의 튜플 생성
    similar_title = als_model.similar_items(title_id, N = 15)
    
    #movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성
    similar_title = [idx_title[i[0]] for i in similar_title]
    
    return similar_title

In [45]:
similar_movie('Men in Black (1997)')

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Matrix, The (1999)',
 'Fifth Element, The (1997)',
 'Face/Off (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Rocky Horror Picture Show, The (1975)',
 'Contact(1997)',
 'Escape from New York (1981)',
 'True Lies (1994)',
 'Bewegte Mann, Der (1994)']

In [46]:
similar_movie('Contact(1997)')

['Contact(1997)',
 'Shopping (1994)',
 "Brother's Kiss, A (1997)",
 'War at Home, The (1996)',
 'Number Seventeen (1932)',
 'Century (1993)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 'Neon Bible, The (1995)',
 "Another Man's Poison (1952)",
 'Killer: A Journal of Murder (1995)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Daens (1992)',
 'Male and Female (1919)',
 'Sweet Nothing (1995)',
 '24 7: Twenty Four Seven (1997)']

In [48]:
similar_movie('Terminator, The (1984)')

['Terminator, The (1984)',
 'Aliens (1986)',
 'Die Hard (1988)',
 'Alien (1979)',
 'Predator (1987)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Robocop (1987)',
 'Blade Runner (1982)',
 'Total Recall (1990)',
 'Indiana Jones and the Last Crusade (1989)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (1981)',
 'Contact(1997)',
 'Star Wars: Episode IV - A New Hope (1977)']

In [49]:
similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 'Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 'Pleasantville (1998)',
 'Contact(1997)',
 "There's Something About Mary (1998)",
 'Mulan (1998)',
 'Tarzan (1999)',
 'Jungle Book, The (1967)',
 'Hercules (1997)']

In [50]:
similar_movie('Forrest Gump (1994)')

['Forrest Gump (1994)',
 'Groundhog Day (1993)',
 'Pretty Woman (1990)',
 'Ghost (1990)',
 'Sleepless in Seattle (1993)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Contact(1997)',
 'Four Weddings and a Funeral (1994)',
 "My Best Friend's Wedding (1997)",
 'As Good As It Gets (1997)',
 'Clueless (1995)',
 'Wedding Singer, The (1998)',
 'Notting Hill (1999)',
 'Doctor Zhivago (1965)',
 'Pleasantville (1998)']

### - 내가 가장 좋아할 만한 영화 추천

In [52]:
user = user_idx['hyojeong']

#recommend에서는 user*item CSR Matrix를 받음.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_title[i[0]], i[1]))

Terminator 2: Judgment Day (1991) 0000.480
Toy Story 2 (1999) 0000.438
Matrix, The (1999) 0000.434
Jurassic Park (1993) 0000.386
Star Wars: Episode VI - Return of the Jedi (1983) 0000.376
Groundhog Day (1993) 0000.340
Bug's Life, A (1998) 0000.318
Total Recall (1990) 0000.309
Star Wars: Episode V - The Empire Strikes Back (1980) 0000.261
Aladdin (1992) 0000.231
Independence Day (ID4) (1996) 0000.226
Aliens (1986) 0000.223
Back to the Future (1985) 0000.206
Star Wars: Episode IV - A New Hope (1977) 0000.206
Lion King, The (1994) 0000.204
Babe (1995) 0000.203
Sixth Sense, The (1999) 0000.203
My Cousin Vinny (1992) 0000.200
Pleasantville (1998) 0000.188
Mission: Impossible (1996) 0000.186


In [53]:
#사용자의 기록(선호하는 영화)이 특정 영화 추천에 기여한 정도
recommended = title_idx['Aliens (1986)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_title[i[0]], i[1]) for i in explain[1]]

[('Terminator, The (1984)', 0.25627567138296864),
 ('Toy Story (1995)', 0.004751556011380804),
 ('Contact(1997)', 0.0006286131795732468),
 ('Forrest Gump (1994)', -0.01479055050346404),
 ('Men in Black (1997)', -0.026166720393109563)]

---

## 프로젝트 정리

### - 데이터 전처리 : 
- csv 파일 2개를 합치는 것이 우선이었다. 평점과 제목이 각각 파일에 따로 있었기 때문에 'movie_id'를 기준으로 merge를 사용하여 한 파일로 합쳤다.    
    
    
### - csr matrix : 
- 노드에서 분명 인덱싱까지 완료된 깔끔하게 정리된 파일이라고 해서 인덱싱 과정이 필요없다고 생각했었는데 csr matrix를 생성할 때 오류가 생겨서 인덱싱 과정을 진행했다. 내가 선호하는 영화를 추가하면서 인덱싱이 되지 않은 데이터들이 생겼기 때문에 인덱싱을 진행해야 했던 것 같다.   
    
      
### - 영화 선호도 : 
- 내가 선호하는 영화 :
  - '토이 스토리'에 대한 선호도가 0.51244164로 생각보다 낮게 나왔다. 
- 내가 선호하는 영화 리스트에 없던 영화 :  
  - '매트릭스'에 대한 선호도는 0.43386862로 '토이 스토리'보다 낮게 나온 걸로 보아 모델이 잘 예측했다고 판단했다.   
   
   
### - 영화 추천 : 
- 내가 선호하는 영화와 비슷한 영화 추천 : 
  - 추천 영화 목록을 보면 그럴듯 한 영화가 꽤 많았다. 뜬금없는 영화도 있긴 했지만 전반적으로 비슷한 장르의 영화를 추천해 준 것 같다. 
- 내가 가장 좋아할 만한 영화 추천 : 
  - 똑똑한 건지 꼼수를 부린건진 몰라도 내가 선호하는 영화의 뒷 시리즈를 추천해 줬다 ㅋㅋ... 시리즈를 제외하고 추천하게끔 했어야 했나 싶지만 그만큼 모델이 예측을 잘했다고도 생각할 수 있을 것 같다.